# Resource management

> Manage project resources such as data and images.

This module serves two main purposes.
- Consolidate paths to resources used across project modules.
- Provide methods to sync resources between local and remote storage locations.

Main parts of the interface, available in module namespace (`import resouces`):
- `paths` dict: paths to project root and remote data repo.
- `Resouce` class: representation of a single file resouce.
- `add`, `get` and `remove` functions: interact with default resouce registry.

Resources are either synced or not. "Sync" resouce has local and remote paths and can be sent both ways. "Non-sync" resource has only single path.

Registered resources are added to a JSON registry file (`resources.json` in project root by default), which is intended to be version controlled by Git.

### Example 1. Refer to file and directory paths

To support portability, use absolute paths defined relative to `paths['root']`. It is encouraged to use `pathlib.Path` objects.

```python
temp_table_path = resources.paths['root'] / 'tmp/tab.csv'
temp_table = pd.read_csv(temp_table_path)
```

### Example 2. Read-only use of resource created and registered by others

Use `resouces.get()` without arguments to see all resources registered in the project. Pass a key argument to obtain a single resouce or a specific subset.

Non-sync resource example.
```python
raw_data_res = resources.get('raw/table/csv')
raw_data_df = pd.read_csv(raw_data_res.path)
```

Sync resource example.
```python
proc_data_res = resouces.get('proc/table/csv')
proc_data_df = pd.read_csv(proc_data_res.remote_path)
```

### Example 3. Pull-modify-push sync resource

Take something created by others, modify it and share the updated version.

```python
proc_data_res = resouces.get('proc/table/csv')
# get local copy
proc_data_res.pull()
# modify and save local copy
proc_data_df = pd.read_csv(proc_data_res.path)
modified = proc_data_df.sort_values()
modified.to_csv(proc_data_res.path)
# share updated version
proc_data_res.push()
```

### Example 4. Create and register new resource

```python
fig_res = resources.Resource(key='out/fig', path='output/fig.png', description='Rise and fall')
resources.add(fig_res)
plt.savefig(fig_res.path)
fig_res.push()
```

In [ ]:
from nbdev import *
%nbdev_default_export resources

In [ ]:
%nbdev_export

import functools
from pathlib import Path
import shutil
import tempfile
import configparser
import json
import pickle
import datetime
import logging
import sys

In [ ]:
%nbdev_export

# it's probably worth organizing this into a Paths class
def _get_root_dir():
    """Return `pathlib.Path` to project root directory.
    Root is identified by presence of "settings.ini" file."""
    d = Path.cwd()
    while d != d.parent:
        if (d / 'settings.ini').exists(): return d
        d = d.parent
    raise Exception(f'Can not find project root at or above "{Path.cwd()}"')
    

def abs_path(path):
    path = Path(path)
    if path.is_absolute():
        return path
    return _get_root_dir() / path


def _get_paths_from_config():
    root = _get_root_dir()
    config = configparser.ConfigParser()
    config.read(root / 'settings.ini')
    paths = dict()
    paths['root'] = root
    
    cache = abs_path(config['resources']['cache_dir'])
    cache.mkdir(parents=True, exist_ok=True)
    paths['cache'] = cache
    
    remote = Path(config['resources']['remote_dir'])
    assert remote.is_absolute()
    assert remote.exists() and remote.is_dir()
    paths['remote'] = remote
    
    return paths


In [ ]:
%nbdev_export
class Resource:
    """Abstraction of a single file."""
    def __init__(self, key, path, description='', sync=True, public=False):
        """Declare new resource. `key` should be a short name.
        Underlying files do not need to exist.
        `sync` enabled resources must have relative `path`, 
        they can be synced between local project directory and remote resource repository.
        """
        self.key = key
        self._raw_path = path
        self.description = description
        self.sync = sync
        self.public = public
        
        path = Path(path)
        if sync:
            assert not path.is_absolute()
            self.path = paths['root'] / path
            self.remote_path = paths['remote'] / path
        else:
            if not path.is_absolute():
                path = paths['root'] / path
            self.path = path
            self.remote_path = None
            
    def __repr__(self):
        return repr(self._to_dict())
    
    __str__ = __repr__

    def _to_dict(self):
        return dict(key=self.key, path=self._raw_path, description=self.description, sync=self.sync, public=self.public)
    
    @classmethod
    def _from_dict(cls, d):
        return cls(**d)
    
    def push(self):
        """Copy sync-enabled resource from local to remote."""
        if not self.sync: 
            print(f'Can not push sync-disabled resource "{self.key}"')
            return
        self.remote_path.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy2(self.path, self.remote_path)
        print(f'Pushed "{self.key}": {self.path} -> {self.remote_path}')
    
    def pull(self):
        """Copy sync-enabled resource from remote to local."""
        if not self.sync: 
            print(f'Can not pull sync-disabled resource "{self.key}"')
            return
        self.path.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy2(self.remote_path, self.path)
        print(f'Pulled "{self.key}": {self.remote_path} -> {self.path}')
    
    def clear(self):
        """Remove local copy of sync-enabled resource."""
        if not self.sync: 
            print(f'Can not clear sync-disabled resource "{self.key}"')
            return
        self.path.unlink()
        print(f'Removed local "{self.key}": {self.path}')
        
    def clear_remote(self):
        """Remove remote copy of sync-enabled resource."""
        if not self.sync: 
            print(f'Can not clear sync-disabled resource "{self.key}"')
            return
        self.remote_path.unlink()
        print(f'Removed remote "{self.key}": {self.path}')
    
    def publish(self):
        """Send resource to public-facing repository."""
        pass

In [ ]:
%nbdev_export
class Registry:
    """Registry of resources, synced with file on disk."""
    def __init__(self, registry_file='resources.json'):
        self.file = abs_path(registry_file)

        self.resources = dict()
        if self.file.exists():
            res_dicts = json.load(self.file.open())
            for res_dict in res_dicts.values():
                res = Resource._from_dict(res_dict)
                self.resources[res.key] = res     
        else:
            self._dump()

    def _dump(self):
        """Save registry to disk.
        Records are ordered by key."""
        # Dicts iterate in the order of key insertions.
        # Assuming that json.dump() maintains that ordering.
        res_dicts = dict()
        for key in sorted(self.resources.keys()):
            res_dicts[key] = self.resources[key]._to_dict()
        json.dump(res_dicts, self.file.open('w'), indent=2)

    def add(self, res, overwrite=False):
        """Add new resource to registry."""
        if not overwrite and res.key in self.resources:
            print(f'Resource "{res.key}" already in the registry, did not overwrite.')
            return
        if res.key in self.resources:
            old_res = self.resources[res.key]
            print(f'Overwriting existing resource: {old_res}')
        self.resources[res.key] = res
        self._dump()
        print(f'Resource "{res.key}" added to registry.')
    
    def remove(self, key):
        """Remove resource with exact `key`."""
        if key not in self.resources:
            print(f'Resource "{key}" not found.')
            return
        del self.resources[key]
        self._dump()
        print(f'Resource "{key}" removed from registry.')
    
    def get(self, key_pattern='*'):
        """Return list of resources with key matching glob-like `key_pattern`.
        If match is unique, return single resource object.
        List is sorted by key.
        """
        matches = [r for k, r in self.resources.items()
                   if Path(k).match(key_pattern)]
        matches.sort(key=lambda res: res.key)
        return matches[0] if len(matches) == 1 else matches
            

## Module interface

In [ ]:
%nbdev_export

paths = _get_paths_from_config()

_registry = Registry('resources.json')

add = _registry.add
get = _registry.get
remove = _registry.remove

def pull(key_pattern='*'):
    """Get resources matching `key_pattern` from remote repository."""
    for r in _registry.get(key_pattern):
        if r.sync:
            r.pull()

def push(key_pattern='*'):
    """Send resources matching `key_pattern` to remote repository."""
    for r in _registry.get(key_pattern):
        if r.sync:
            r.push()

## Caching

In [ ]:
%nbdev_export
def _hash(x):
    """Generalized version of builtin hash() to support some unhashable types."""
    # Recursive calls allow to hash nested structures.
    import pandas as pd
    
    if isinstance(x, (pd.Series, pd.DataFrame)):
        # WARNING: only head of the dataframe is hashed, for speed sake
        x_head_hash_series = pd.util.hash_pandas_object(x.head())
        return hash(tuple(x_head_hash_series))

    if isinstance(x, dict):
        return _hash(tuple(x.items()))
    
    try:
        return hash(x)
    except TypeError:
        return hash(tuple(_hash(y) for y in x))
        

def hash_args(*args, **kwargs):
    """Return hash of passed arguments."""
    return _hash(args + tuple(kwargs.items()))

def func_sig(fname, *args, **kwargs):
    """Return string representation of function call signature."""
    import pandas as pd
    def _repr(x):
        if isinstance(x, pd.Series):
            return f'series({len(x)})'
        if isinstance(x, pd.DataFrame):
            return f'dataframe{x.shape}'
        return repr(x)
    
    ar = ', '.join(_repr(x) for x in args)
    kw = ', '.join(f'{k}: {_repr(v)}' for k, v in kwargs.items())
    arkw = ', '.join(x for x in [ar, kw] if x != '')
    return f'{fname}({arkw})'

def cacheable(func):
    """Enable caching of function results."""
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        arghash = hash_args(*args, **kwargs)
        fn = func.__name__
        sig = func_sig(fn, *args, **kwargs)
        cached = paths['cache'] / fn / str(arghash)
        
        if cached.exists():
            modified = datetime.datetime.fromtimestamp(cached.stat().st_mtime).isoformat(' ')
            logging.info(f'Return cached result of {sig}, last modified at {modified}')
            return pickle.load(cached.open('rb'))

        result = func(*args, **kwargs)
        cached.parent.mkdir(exist_ok=True)
        pickle.dump(result, cached.open('wb'))
        logging.info(f'Result of {sig} cached to {cached}')
        return result
            
    return wrapper

## Tests

In [ ]:
# TEST: Convert resource to and from dict.
d = dict(key='a/b', path='a/b.csv', description='data B', sync=True, public=True)
r = Resource._from_dict(d)
assert r._to_dict() == d

In [ ]:
# TEST: Pull synced resource, update content and push back, clean up.
r = Resource('test/pull-upd-push', 'tmp/pull-upd-push.txt', 'Test pull-upd-push')
r.remote_path.parent.mkdir(parents=True, exist_ok=True)
r.remote_path.write_text('Jack')
r.pull()
assert r.path.read_text() == 'Jack'
f = r.path.open('a')
f.write(' and Jill')
f.close()
r.push()
assert r.remote_path.read_text() == 'Jack and Jill'
r.clear()
assert not r.path.exists()
r.clear_remote()
assert not r.remote_path.exists()

In [ ]:
# TEST: Add, get and remove resources in registry.
r = Registry('test_resources.json')
assert r.get() == []
r.add(Resource('in/data', 'input/data.csv', 'input data'))
r.add(Resource('out/data', 'output/result.csv', 'output data'))
assert [x.key for x in r.get()] == ['in/data', 'out/data']
assert r.get('out/data').description == 'output data'
r.remove('out/data')
assert r.get().description == 'input data'
r.add(Resource('out/data', 'output/result.csv', 'output data'))
r.add(Resource('out/fig', 'output/image.png', 'output figure'))
assert [x.key for x in r.get()] == ['in/data', 'out/data', 'out/fig']
assert [x.description for x in r.get('*/data')] == ['input data', 'output data']
assert [x.description for x in r.get('out/*')] == ['output data', 'output figure']
assert r.get('out/???').description == 'output figure'
assert r.get('whole/world') == []
r.remove('in/data')
r.remove('out/data')
r.remove('out/fig')
assert r.get() == []
assert r.file.read_text() == '{}'
r.file.unlink()

In [ ]:
# TEST: caching
hash_args(14, 'hello', None, [1, 'a'], ([1, 2], 3), {'a': 1, 'b': 2}, key1=12, key2='ab')
hash_args(pd.Series([1, 'a', None]), pd.DataFrame([[1,2],[3,4]]))

@cacheable
def quack(who):
    "Who quacks?"
    return f'{who} quacks'

quack('he')